## Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## Data Load

In [2]:
apply_train_df = pd.read_csv('apply_train.csv')

In [12]:
apply_train_df.info()
#resume_seq = 구직자
#recruitment_seq = 채용 공고

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57946 entries, 0 to 57945
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   resume_seq       57946 non-null  object
 1   recruitment_seq  57946 non-null  object
dtypes: object(2)
memory usage: 905.5+ KB


## User-Item Matrix / Similarity / Score

In [14]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
## .groupby : 두 컬럼을 기준으로 그룹화 한다. 즉, 각 구직자가 어떤 채용공고에 지원했는가를 나타낸다.
## .size : 각 그룹의 크기를 계산한다. 즉, 몇 번 지원했는지를 나타낸다.
## .unstack(fill_value=0) : 그룹 데이터를 Unpivot하고, 결측치(NaN)은 0으로 채운다. 
user_item_matrix[user_item_matrix > 1] = 1
## 1보다 크면 1로 변환하다. 즉, 중복 지원 제거.

# 사용자 간의 유사성 계산
## 유사성 계산을 왜하지? -> 다른 사용자의 행동을 기반으로 추천을 수행하기 위해서.
## 여기서 사용된 유사성 지표는 코사인 유사성이다.
user_similarity = cosine_similarity(user_item_matrix)
## cosine_similarity는 입력으로 구직자-공고 행렬(user_item_matrix)을 받아 사용자 간의 코사인 유사성을 계산
## 두 벡터(사용자) 간의 각도에 대한 코사인 값을 사용하여 유사성을 측정하는 지표
## 두 사용자 간의 코사인 유사성이 높을수록 그들의 관심사가 유사하다고 판단 (1에 가까울수록 유사하고 -1에 가까울 수록 상반됨)
### 즉, 어떤 유저들끼리 비슷한 공고들에 지원했다면, 유사도가 높을 것이다.

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T
## .dot : user_similarity(유사성) 매트릭스와 user_item_matrix(지원 여부)를 행렬곱(내적)한다. 즉, 구직자-공고 행렬에 유사성을 활용한 가중치가 적용된다.
## .abs : 절대값을 취한다. why? 상반의 관계라도 있기 때문에 적용하는걸까? 잘 모르겠다.
## .sum(axis=1) : 각 행(구직자)을 따라 더한다. shape이 [n, 1] 형태를 띤다. 즉, 한 구직자가 다른 구직자들과 얼마나 유사한지를 나타내는 정도가 된다.
## np.array를 씌워서 [1, n] 형태의 행 벡터로 전환한다.
## .T : 행 벡터를 전치(transpose)한다. [n, 1] 형태가 된다. 즉, 열 벡터 형태를 띄게 된다.

## 따라서 최종 결과값은 : 가중치가 합산된 구직자-공고 매트릭스를 구직자간 유사도로 나눈다. 
## why? 채용 공고에 대한 예측 점수를 정규화 된 유사성을 사용하여 모든 사용자의 예측 점수를 [0, 1] 범위로 조정하기 위함이다.

In [21]:
user_item_matrix.info()
# 총 지원자 수 : 8482명
# 총 채용공고 : 6695개

<class 'pandas.core.frame.DataFrame'>
Index: 8482 entries, U00001 to U08482
Columns: 6695 entries, R00001 to R06695
dtypes: int64(6695)
memory usage: 433.3+ MB


In [19]:
user_similarity[0], len(user_similarity[0])
#유저간 매트릭스니까 당연히 유사도 결과값도 8482의 길이를 가짐

(array([1., 0., 0., ..., 0., 0., 0.]), 8482)

In [25]:
user_similarity[:5]
'''
1: 두 벡터가 완전히 동일한 방향을 가질 때
0: 두 벡터가 독립적이며 관련이 없을 때
-1: 두 벡터가 완전히 상반된 방향을 가질 때
'''

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
len(user_predicted_scores), len(user_predicted_scores[0])

(8482, 6695)

## Prediction

In [4]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)
    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
    for job in recommended_jobs:
        recommendations.append([user, job])

## Submission

In [5]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./baseline_submit.csv', index=False)